[View in Colaboratory](https://colab.research.google.com/github/aminzabardast/Tensorflow-Tutorials/blob/dev/5_Convolutional_Neural_Network.ipynb)

# Convolutional Neural Networks With Tensor Flow

**Convolutional Neural Networks**, also known as **ConvNet**s or **CNN**s, are a type of Neural Networks that performe better in specific data types. These type of networks are specially better for image analysis.

This notebook is dedicated to creating CNNs using Tensoflow library and the specific example that will be studied is [MNIST database](http://yann.lecun.com/exdb/mnist/) for handwritten digit recognition.

In [1]:
!pip show tensorflow
!rm -rf logs

Name: tensorflow
Version: 1.9.0rc1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: gast, six, setuptools, grpcio, wheel, termcolor, protobuf, absl-py, tensorboard, numpy, astor
Required-by: 


## Loading Libraries and Data

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

## Function: Divide and Conquer

In [4]:
def conv2d(input_data, kernel_shape, activation_function=tf.nn.relu, name=None):
    weights = tf.Variable(tf.truncated_normal(kernel_shape, stddev=0.1))
    biases = tf.Variable(tf.constant(.1, shape=[kernel_shape[3]]))
    conv_result = tf.nn.conv2d(input_data, weights, strides=[1,1,1,1], padding='SAME', name=name) + biases
    return activation_function(conv_result)

In [5]:
def maxPool2x2(input_data, name=None):
    return tf.nn.max_pool(input_data, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

In [6]:
def fullyConnectedLayer(input_data, shape, activation_function=tf.nn.relu, name=None):
    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    biases = tf.Variable(tf.zeros([1, shape[1]]) + 0.1)
    return activation_function(tf.matmul(input_data, weights) + biases)

In [7]:
def compute_accuracy(v_xs, v_ys):
    y_pre = sess.run(rfc2, feed_dict={xs: v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result

## Creating The Network and Training

In [8]:
with tf.name_scope('Input'):
    xs = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='Data')/255
    x_images = tf.reshape(xs,shape=[-1, 28, 28, 1], name='ReshapedData')
with tf.name_scope('Truth'):
    ys = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='Truth')

with tf.name_scope('Conv1'):
    kernel1_shape = [5, 5, 1, 32]  # 5x5 Kernel, Feature Map Input 1 (Gray scale image) and Output 32
    rconv1 = conv2d(x_images, kernel1_shape)  # Using kernel1 to calculate the convolved layer / Output size: 28 x 28 x 32

with tf.name_scope('MaxPool1'):
    rpool1 = maxPool2x2(rconv1)  # Max polling the result of Conv1 layer / Output size: 14 x 14 x 32

with tf.name_scope('Conv2'):
    kernel2_shape = [3, 3, 32, 64]  # 3x3 Kernel, Feature Map Input 32 and Output 64
    rconv2 = conv2d(rpool1, kernel2_shape)  # Using kernel2 to calculate the convolved layer / Output size: 14 x 14 x 64

with tf.name_scope('MaxPool2'):
    rpool2 = maxPool2x2(rconv2)  # Max polling the result of Conv2 layer / Output size: 7 x 7 x 64

with tf.name_scope('FC1'):
    fc1_shape = [7*7*64, 1024]  # Shape of the layer
    rpool2_flattened = tf.reshape(rpool2, shape=[-1, 7*7*64])  # From [n,7,7,64] to [n,3136]
    rfc1 = fullyConnectedLayer(rpool2_flattened, fc1_shape)

with tf.name_scope('FC2'):
    fc2_shape = [1024, 10]  # Shape of th layer
    rfc2 = fullyConnectedLayer(rfc1, fc2_shape, tf.nn.softmax)

with tf.name_scope('Loss'):
    crossEntropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(rfc2), reduction_indices=[1]))
    tf.summary.scalar(name='CrossEntropy', tensor=crossEntropy)

with tf.name_scope('Optimizer'):
    trainStep = tf.train.AdamOptimizer().minimize(crossEntropy)

sess = tf.Session()

init = tf.global_variables_initializer()

merged = tf.summary.merge_all()

trainWriter = tf.summary.FileWriter(logdir='logs/train', graph=sess.graph)
testWriter = tf.summary.FileWriter(logdir='logs/test', graph=sess.graph)

sess.run(init)

for epoch in range(2000):
    trainBatchXs, trainBatchYs = mnist.train.next_batch(100)
    testBatchXs, testBatchYs = mnist.test.next_batch(100)
    
    sess.run(trainStep, feed_dict={xs: trainBatchXs, ys: trainBatchYs})
    
    trainWriter.add_summary(sess.run(merged, feed_dict={xs: trainBatchXs, ys: trainBatchYs}), epoch)
    testWriter.add_summary(sess.run(merged, feed_dict={xs: testBatchXs, ys: testBatchYs}), epoch)
    
    if (epoch+1) % 100 == 0 or epoch == 0:
        print('Epoch: {}'.format(epoch+1))

accuracy = compute_accuracy(mnist.test.images, mnist.test.labels)*100
print('\nFinal Accuracy: %{0:.2f}'.format(accuracy))


Epoch: 1
Epoch: 100
Epoch: 200
Epoch: 300
Epoch: 400
Epoch: 500
Epoch: 600
Epoch: 700
Epoch: 800
Epoch: 900
Epoch: 1000
Epoch: 1100
Epoch: 1200
Epoch: 1300
Epoch: 1400
Epoch: 1500
Epoch: 1600
Epoch: 1700
Epoch: 1800
Epoch: 1900
Epoch: 2000

Final Accuracy: %98.77


## Downloading Log

In [9]:
!tar czvf logs.tar.gz logs

from google.colab import files
files.download('logs.tar.gz')

!rm -rvf logs*

logs/
logs/train/
logs/train/events.out.tfevents.1530557341.792ba13cd039
logs/test/
logs/test/events.out.tfevents.1530557341.792ba13cd039
removed 'logs/train/events.out.tfevents.1530557341.792ba13cd039'
removed directory 'logs/train'
removed 'logs/test/events.out.tfevents.1530557341.792ba13cd039'
removed directory 'logs/test'
removed directory 'logs'
removed 'logs.tar.gz'
